In [2]:
import os
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
data_path = os.path.join('../')
filename = 'nu6n.s3db'

In [5]:
sqliteConnection = sqlite3.connect(os.path.join(data_path, filename))

In [10]:
cursor = sqliteConnection.cursor()
print('DB Init')

DB Init


In [11]:
query = 'select sqlite_version();'
cursor.execute(query)
result = cursor.fetchall()
print('SQLite Version is {}'.format(result))

SQLite Version is [('3.45.3',)]


In [99]:
#find all the tables and their schemas
query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor.execute(query)
tables = cursor.fetchall()
print(tables)
print(cursor.execute("PRAGMA table_info('Contest')").fetchall())
for tbl in tables:
    print(tbl[0], ': ', cursor.execute(f"PRAGMA table_info('{tbl[0]}')").fetchall())

[('Contest',), ('ContestInstance',), ('DBInfo',), ('DBVersionV2',), ('DXLOG',), ('Station',), ('Skeds',), ('sqlite_stat1',), ('sqlite_stat4',)]
[(0, 'Name', 'NVARCHAR(10)', 1, None, 1), (1, 'DisplayName', 'NVARCHAR(50)', 0, None, 0), (2, 'CabrilloName', 'NVARCHAR(15)', 1, None, 0), (3, 'Mode', 'NVARCHAR(6)', 1, None, 0), (4, 'DupeType', 'SMALLINT', 0, "'''''''0'''''''", 0), (5, 'Multiplier1Name', 'NVARCHAR(15)', 0, "'''''''''''''''N/A'''''''''''''''", 0), (6, 'Multiplier2Name', 'NVARCHAR(15)', 0, "'''''''''''''''N/A'''''''''''''''", 0), (7, 'Period', 'INT', 1, "'''''''2'''''''", 0), (8, 'PointsPerContact', 'INT', 0, "'''''''0'''''''", 0), (9, 'Multiplier3Name', 'NVARCHAR(15)', 0, None, 0), (10, 'MasterDTA', 'NVARCHAR(255)', 0, None, 0), (11, 'CWMessages', 'NVARCHAR(255)', 0, None, 0), (12, 'SSBMessages', 'NVARCHAR(255)', 0, None, 0), (13, 'DigiMessages', 'NVARCHAR(255)', 0, None, 0), (14, 'CabrilloVersion', 'NVARCHAR(20)', 0, None, 0)]
Contest :  [(0, 'Name', 'NVARCHAR(10)', 1, None, 1

In [100]:
#fetch a few call for verification
query = """ SELECT Call from DXLOG"""
cursor.execute(query).fetchmany(10)

[('AE4ED',),
 ('AH6KO',),
 ('K2TD',),
 ('K3TV',),
 ('K4WOP',),
 ('K5DDS',),
 ('K6RB',),
 ('KE6JAC',),
 ('KF3P',),
 ('N0HJZ',)]

In [101]:
# convert to pandas dataframe
tbl = tables[4][0]
print(tbl)
q = f'select * from {tbl}'
df = pd.read_sql_query(q, sqliteConnection, index_col='TS', parse_dates='TS')
df.head()

DXLOG


,Call,Freq,QSXFreq,Mode,ContestName,SNT,RCV,CountryPrefix,StationPrefix,QTH,...,GridSquare,Operator,Continent,RoverLocation,RadioInterfaced,NetworkedCompNr,NetBiosName,IsOriginal,ID,CLAIMEDQSO
TS,,,,,,,,,,,,,,,,,,,,,
2022-12-10 06:20:50,K6RB,28035.879,28035.879,CW,ARRL10M,599,599,K,NU6N,,...,,NU6N,NA,,1,0,MIGHTYDUCK,1,bedba7a6d1fc45ee8517c7e1f1122e23,1
2022-12-10 06:21:35,K5DDS,28035.879,28035.879,CW,ARRL10M,599,599,K,NU6N,,...,,NU6N,NA,,1,0,MIGHTYDUCK,1,e9ea2abd27d148b7a126c5e436aa05cc,1
2022-12-10 06:22:06,K3TV,28035.879,28035.879,CW,ARRL10M,599,599,K,NU6N,,...,,NU6N,NA,,1,0,MIGHTYDUCK,1,64d9aaaf08b5445a9fa5f0a0a78e0036,1
2022-12-10 06:22:46,N6TV,28035.879,28035.879,CW,ARRL10M,599,599,K,NU6N,,...,,NU6N,NA,,1,0,MIGHTYDUCK,1,09ce512797ab4355b84b0aeb22d7eea7,1
2022-12-10 17:49:24,K9XR,28095.864,28095.864,CW,ARRL10M,599,599,K,NU6N,,...,,NU6N,NA,,1,0,MIGHTYDUCK,1,b07d4a3d0d2847e4b5992920945eff07,1


In [102]:
arrl10 = df.loc['ContestName']
df.index.max() - df.index.min()

KeyError: 'ContestName'

In [119]:
# check stats about df
df.columns
df.groupby(['ContestName', 'ContestNR'])['ContestNR'].value_counts()
q = """ SELECT * from DXLOG 
where ContestName='ARRL10M'
and ContestNR<0;"""
arrl10m_1 = pd.read_sql_query(q, sqliteConnection, index_col='TS', parse_dates='TS')
arrl10m_1.drop(['Operator', 'StationPrefix', 'QTH', 'GridSquare', 'Name',
       'Continent', 'RoverLocation', 'RadioInterfaced', 'NetworkedCompNr', 'ID', 'ContactType', 'Comment',
       'NetBiosName', 'IsOriginal', 'CLAIMEDQSO', 'WPXPrefix', 'RadioNR', 'Power', 'Band'], axis=1, inplace=True)
arrl10m_1

,Call,Freq,QSXFreq,Mode,ContestName,SNT,RCV,CountryPrefix,NR,Sect,...,SentNr,Points,IsMultiplier1,IsMultiplier2,Exchange1,ContestNR,isMultiplier3,MiscText,IsRunQSO,Run1Run2
TS,,,,,,,,,,,,,,,,,,,,,
2022-12-11 16:34:08,AE4ED,28067.270,28067.270,CW,ARRL10M,599,599,K,0,AL,...,211,4,0,0,AL,-1,0,,0,2
2022-12-10 06:20:50,K6RB,28035.879,28035.879,CW,ARRL10M,599,599,K,0,CA,...,1,4,1,0,CA,-1,0,,1,2
2022-12-10 06:21:35,K5DDS,28035.879,28035.879,CW,ARRL10M,599,599,K,0,CA,...,2,4,0,0,CA,-1,0,,1,2
2022-12-10 06:22:46,N6TV,28035.879,28035.879,CW,ARRL10M,599,599,K,0,CA,...,4,4,0,0,CA,-1,0,,0,2
2022-12-10 06:22:06,K3TV,28035.879,28035.879,CW,ARRL10M,599,599,K,0,GA,...,3,4,1,0,GA,-1,0,,0,2
2022-12-10 18:48:20,N0HJZ,28095.508,28095.508,CW,ARRL10M,599,599,K,0,MN,...,34,4,0,0,MN,-1,0,,0,2
2022-12-10 18:49:08,N0HJZ,28095.508,28095.508,CW,ARRL10M,599,599,K,0,MN,...,34,4,0,0,MN,-1,0,,0,2


In [125]:
q = """ SELECT * from DXLOG 
where ContestName='ARRL10M'
and (Call='AE4ED' or Call='K6RB' or Call='K5DDS' or Call='N6TV' or Call='K3TV' or Call='N0HJZ')
and ContestNR>0;"""
arrl10m = pd.read_sql_query(q, sqliteConnection, index_col='TS', parse_dates='TS')
arrl10m.drop(['Operator', 'StationPrefix', 'QTH', 'GridSquare', 'Name',
       'Continent', 'RoverLocation', 'RadioInterfaced', 'NetworkedCompNr', 'ID', 'ContactType', 'Comment',
       'NetBiosName', 'IsOriginal', 'CLAIMEDQSO', 'WPXPrefix', 'RadioNR', 'Power', 'Band'], axis=1, inplace=True)
arrl10m

,Call,Freq,QSXFreq,Mode,ContestName,SNT,RCV,CountryPrefix,NR,Sect,...,SentNr,Points,IsMultiplier1,IsMultiplier2,Exchange1,ContestNR,isMultiplier3,MiscText,IsRunQSO,Run1Run2
TS,,,,,,,,,,,,,,,,,,,,,
2022-12-11 16:36:12,AE4ED,28067.270,28067.270,CW,ARRL10M,599,599,K,0,AL,...,213,4,0,0,AL,1,0,,0,2
2022-12-10 18:50:45,N0HJZ,28095.508,28095.508,CW,ARRL10M,599,599,K,0,MN,...,34,4,0,0,MN,1,0,,0,2
2022-12-10 18:39:08,N6TV,28005.336,28005.336,CW,ARRL10M,599,599,K,0,CA,...,31,4,0,0,CA,1,0,,0,2


In [ ]:
df.groupby(['Contest_ID']).value_counts()

KeyError: 'Contest_ID'

In [ ]:
if sqliteConnection:
    sqliteConnection.close()
    print('SQLite Connection closed')

In [14]:
s = pd.Series([12, 22, 32], index=['x', 'y', 'z'])
sm = pd.Series({'d': 33, 'b': 44})
calories = {"day1": 420, "day2": 380, "day3": 390}

myvar = pd.Series(calories)

print(myvar['day1'])

pd.options.display.max_rows

420


60

In [19]:
json_split = '''
{
    "columns": ["column1", "column2"],
    "index": [0, 1, 2],
    "data": [[1, "a"], [2, "b"], [3, "c"]]
}
'''
df = pd.read_json(json_split, orient='split')
print(df)


   column1 column2
0        1       a
1        2       b
2        3       c


/var/folders/v9/_2d84cyn3l5_yp66990l3c040000gn/T/ipykernel_2263/3885762115.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_split, orient='split')


In [20]:
json_records = '''
[
    {"column1": 1, "column2": "a"},
    {"column1": 2, "column2": "b"},
    {"column1": 3, "column2": "c"}
]
'''
df = pd.read_json(json_records, orient='records')
print(df)


   column1 column2
0        1       a
1        2       b
2        3       c


/var/folders/v9/_2d84cyn3l5_yp66990l3c040000gn/T/ipykernel_2263/977280915.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_records, orient='records')
